In [5]:
#all the imports
from keras.layers.core import Activation, Dense, Dropout, SpatialDropout1D, RepeatVector
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import collections
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os

Using TensorFlow backend.


In [64]:
DATA_DIR="/Users/tanaykarmarkar/nltk_data"
fedata=open(os.path.join(DATA_DIR,"treebank_sents.txt"),"w")#w will be used instead of wb
ffdata=open(os.path.join(DATA_DIR,"treebank_poss.txt"),"w")
sents=nltk.corpus.treebank.tagged_sents()
for sent in sents:
    words, poss=[],[]
    for word, pos in sent:
        if pos=="-NONE-":
            continue
        words.append(word)
        poss.append(pos)
    fedata.write('{:s}n'.format(' ').join(words))
    ffdata.write("{:s}n".format(' ').join(poss))
fedata.close()
ffdata.close()
print(words)

['Trinity', 'said', 'it', 'plans', 'to', 'begin', 'delivery', 'in', 'the', 'first', 'quarter', 'of', 'next', 'year', '.']


In [61]:
fedata

<_io.TextIOWrapper name='/Users/tanaykarmarkar/nltk_data/treebank_sents.txt' mode='w' encoding='UTF-8'>

In [65]:
sents

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], ...]

In [63]:
ffdata

<_io.TextIOWrapper name='/Users/tanaykarmarkar/nltk_data/treebank_poss.txt' mode='w' encoding='UTF-8'>

In [66]:
words

['Trinity',
 'said',
 'it',
 'plans',
 'to',
 'begin',
 'delivery',
 'in',
 'the',
 'first',
 'quarter',
 'of',
 'next',
 'year',
 '.']

In [28]:
sents

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], ...]

In [15]:
def parse_sentences(filename):
    words_freqs=collections.Counter()
    num_recs, maxlen=0,0
    fin=open(filename,"rb")
    for line in fin:
        words=line.strip().lower().split()
        for word in words:
            words_freqs[word]+=1
        if len(words)>maxlen:
            maxlen=len(words)
        num_recs+=1
    fin.close()
    return words_freqs,maxlen,num_recs
s_wordfreqs,s_maxlen,s_numrecs=parse_sentences(os.path.join(DATA_DIR,"treebank_sents.txt"))
t_wordfreqs,t_maxlen,t_numrecs=parse_sentences(os.path.join(DATA_DIR,"treebank_poss.txt"))
print(len(s_wordfreqs),s_maxlen,s_numrecs,len(t_wordfreqs),t_maxlen,t_numrecs)

11863 90171 1 150 90171 1


In [58]:
s_word2index["UNK"]

1

In [17]:
MAX_SEQLEN=250
S_MAX_FEATURES=5000
T_MAX_FEATURES=45

In [18]:
s_vocabsize=min(len(s_wordfreqs), S_MAX_FEATURES)+2
s_word2index={x[0]:i+2 for i,x in enumerate(s_wordfreqs.most_common(S_MAX_FEATURES))}
s_word2index["PAD"]=0
s_word2index["UNK"]=1
s_index2word={v:k for k,v in s_word2index.items()}

In [39]:
t_vocabsize=len(t_wordfreqs)+1
t_word2index={x[0]:i for i,x in enumerate(t_wordfreqs.most_common(T_MAX_FEATURES))}

In [40]:
s_maxlen

90171

In [38]:
s_wordfreqs

Counter({b'pierre': 1,
         b'nvinken': 2,
         b'n,': 4883,
         b'n61': 5,
         b'nyears': 115,
         b'nold': 31,
         b'nwill': 281,
         b'njoin': 4,
         b'nthe': 4111,
         b'nboard': 73,
         b'nas': 385,
         b'na': 1896,
         b'nnonexecutive': 5,
         b'ndirector': 32,
         b'nnov.': 24,
         b'n29': 5,
         b'n.mr.': 103,
         b'nis': 672,
         b'nchairman': 56,
         b'nof': 2322,
         b'nelsevier': 1,
         b'nn.v.': 3,
         b'ndutch': 3,
         b'npublishing': 17,
         b'ngroup': 69,
         b'n.rudolph': 1,
         b'nagnew': 1,
         b'n55': 10,
         b'nand': 1515,
         b'nformer': 19,
         b'nconsolidated': 1,
         b'ngold': 11,
         b'nfields': 13,
         b'nplc': 13,
         b'nwas': 367,
         b'nnamed': 22,
         b'nthis': 196,
         b'nbritish': 11,
         b'nindustrial': 25,
         b'nconglomerate': 3,
         b'n.a': 81,
         b

In [35]:
s_vocabsize

5002

In [33]:
s_word2index

{b'n,': 2,
 b'nthe': 3,
 b'nof': 4,
 b'nto': 5,
 b'na': 6,
 b'nin': 7,
 b'nand': 8,
 b"n's": 9,
 b'nfor': 10,
 b'nthat': 11,
 b'n$': 12,
 b'nis': 13,
 b'nsaid': 14,
 b'n.the': 15,
 b'nit': 16,
 b'non': 17,
 b"n''": 18,
 b'n%': 19,
 b'nby': 20,
 b'n``': 21,
 b'nat': 22,
 b'nwith': 23,
 b'nfrom': 24,
 b'nas': 25,
 b'nmillion': 26,
 b'nare': 27,
 b'nwas': 28,
 b'nbe': 29,
 b'nhas': 30,
 b'nits': 31,
 b'nhave': 32,
 b"nn't": 33,
 b'nan': 34,
 b'nnew': 35,
 b'nor': 36,
 b'nwill': 37,
 b'n.``': 38,
 b'n.': 39,
 b'ncompany': 40,
 b'nmr.': 41,
 b'nhe': 42,
 b'nwhich': 43,
 b'nthey': 44,
 b'nu.s.': 45,
 b'nyear': 46,
 b'nsays': 47,
 b'nwould': 48,
 b'n--': 49,
 b'nabout': 50,
 b'nmore': 51,
 b'nwere': 52,
 b'nthis': 53,
 b'nmarket': 54,
 b'nthan': 55,
 b'ntheir': 56,
 b'n.in': 57,
 b'n;': 58,
 b'nstock': 59,
 b'nhad': 60,
 b'nwho': 61,
 b'ntrading': 62,
 b'nbillion': 63,
 b'npresident': 64,
 b'nbut': 65,
 b'nup': 66,
 b'nbeen': 67,
 b'nalso': 68,
 b'n.but': 69,
 b'none': 70,
 b'nother': 71,
 b'

In [22]:
t_vocabsize

151

In [23]:
t_word2index

{b'n$': 22,
 b"n''": 26,
 b'n,': 6,
 b'n-rrb-': 44,
 b'n.': 31,
 b'n.``': 30,
 b'n.cc': 36,
 b'n.dt': 19,
 b'n.in': 25,
 b'n.jj': 43,
 b'n.nn': 40,
 b'n.nnp': 23,
 b'n.nns': 39,
 b'n.prp': 34,
 b'n.rb': 38,
 b'n:': 24,
 b'n``': 28,
 b'ncc': 14,
 b'ncd': 7,
 b'ndt': 3,
 b'nin': 1,
 b'njj': 5,
 b'njjr': 29,
 b'njjs': 37,
 b'nmd': 18,
 b'nnn': 0,
 b'nnnp': 2,
 b'nnnps': 32,
 b'nnns': 4,
 b'npos': 20,
 b'nprp': 15,
 b'nprp$': 21,
 b'nrb': 9,
 b'nrbr': 42,
 b'nrp': 35,
 b'nto': 11,
 b'nvb': 10,
 b'nvbd': 8,
 b'nvbg': 16,
 b'nvbn': 13,
 b'nvbp': 17,
 b'nvbz': 12,
 b'nwdt': 27,
 b'nwp': 33,
 b'nwrb': 41}

In [55]:
def build_tensor(filename,numrecs,word2index,maxlen,make_categorical=False,num_classes=0):
    data=np.empty(numrecs,dtype=list)
    fin=open(filename,"r")
    i=0
    for line in fin:
        wids=[]
        for word in line.strip().lower().split():
            if word in word2index:#has_key was removed in python
                wids.append(word2index[word])
            else:wids.append(word2index['UNK'])
        if make_categorical:
            data[i]=np_utils.to_categorical(wids,num_classes=num_classes)
        else:
            data[i]=wids
        i+=1
    fin.close()
    pdata=sequence.pad_sequences(data, maxlen=maxlen)
    return pdata

X=build_tensor(os.path.join(DATA_DIR, "treebank_sents.txt"),s_numrecs,s_word2index,MAX_SEQLEN,False,s_vocabsize)
Y=build_tensor(os.path.join(DATA_DIR, "treebank_poss.txt"),t_numrecs,t_word2index,MAX_SEQLEN, True, t_vocabsize)


                
            
        
    


KeyError: 'UNK'

In [51]:
s_word2index

{b'n,': 2,
 b'nthe': 3,
 b'nof': 4,
 b'nto': 5,
 b'na': 6,
 b'nin': 7,
 b'nand': 8,
 b"n's": 9,
 b'nfor': 10,
 b'nthat': 11,
 b'n$': 12,
 b'nis': 13,
 b'nsaid': 14,
 b'n.the': 15,
 b'nit': 16,
 b'non': 17,
 b"n''": 18,
 b'n%': 19,
 b'nby': 20,
 b'n``': 21,
 b'nat': 22,
 b'nwith': 23,
 b'nfrom': 24,
 b'nas': 25,
 b'nmillion': 26,
 b'nare': 27,
 b'nwas': 28,
 b'nbe': 29,
 b'nhas': 30,
 b'nits': 31,
 b'nhave': 32,
 b"nn't": 33,
 b'nan': 34,
 b'nnew': 35,
 b'nor': 36,
 b'nwill': 37,
 b'n.``': 38,
 b'n.': 39,
 b'ncompany': 40,
 b'nmr.': 41,
 b'nhe': 42,
 b'nwhich': 43,
 b'nthey': 44,
 b'nu.s.': 45,
 b'nyear': 46,
 b'nsays': 47,
 b'nwould': 48,
 b'n--': 49,
 b'nabout': 50,
 b'nmore': 51,
 b'nwere': 52,
 b'nthis': 53,
 b'nmarket': 54,
 b'nthan': 55,
 b'ntheir': 56,
 b'n.in': 57,
 b'n;': 58,
 b'nstock': 59,
 b'nhad': 60,
 b'nwho': 61,
 b'ntrading': 62,
 b'nbillion': 63,
 b'npresident': 64,
 b'nbut': 65,
 b'nup': 66,
 b'nbeen': 67,
 b'nalso': 68,
 b'n.but': 69,
 b'none': 70,
 b'nother': 71,
 b'